In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import pairwise_distances

# Define constants
IMAGE_SIZE = (224, 224)  # Adjusted to fit MobileNetV2 requirements
BATCH_SIZE = 32
EMBEDDING_SIZE = 128
NUM_CLASSES = len(os.listdir("C:/Users/Plaksha/Desktop/fma_img/fma_img"))

# Function to create a Siamese network model
def create_siamese_model(input_shape, embedding_size):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(embedding_size, activation='relu')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Function to calculate angular loss
def angular_loss(y_true, y_pred):
    loss = 4 * tf.reduce_mean(tf.math.square(tf.norm(y_pred, axis=1) - 1))
    return loss

# Function to extract embeddings from images
def extract_embeddings(directory):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        directory,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='sparse',
        shuffle=False
    )
    num_samples = generator.samples
    model = create_siamese_model((*IMAGE_SIZE, 3), EMBEDDING_SIZE)
    model.compile(optimizer='adam', loss=angular_loss)

    embeddings = model.predict(generator, verbose=1)
    class_labels = generator.classes
    class_names = list(generator.class_indices.keys())

    return embeddings, class_labels, class_names

# Main function
def main():
    embeddings, class_labels, class_names = extract_embeddings("C:/Users/Plaksha/Desktop/fma_img/fma_img")

    # Separate embeddings by class labels
    class_embeddings = {}
    for class_label in range(NUM_CLASSES):
        class_embeddings[class_label] = []

    for embedding, label in zip(embeddings, class_labels):
        class_embeddings[label].append(embedding)

    # Print embeddings for each class
    for class_label, embeddings in class_embeddings.items():
        print(f"Class: {class_label}")
        for i, embedding in enumerate(embeddings, start=1):
            print(f"Image {i} - Embedding Vector: {embedding}")
        class_embeddings_array = np.array(embeddings)
        class_embeddings_tensor = tf.convert_to_tensor(class_embeddings_array)
        loss = angular_loss(None, class_embeddings_tensor)
        print(f"Class {class_label} Loss: {loss.numpy()}")

if __name__ == "__main__":
    main()


Found 250 images belonging to 10 classes.
8/8 [==============================] - 6s 655ms/step
Class: 0
Image 1 - Embedding Vector: [2.6347349  0.93715614 0.         0.         0.         0.11663021
 1.4176815  0.         0.         1.0617663  0.         0.
 1.6782315  0.56673265 0.         1.2984552  0.         0.37131658
 1.090415   0.7576771  0.         0.         0.14860277 0.
 0.         0.94293153 0.4728084  0.         0.         0.
 0.70414877 0.7399719  0.8743497  0.13495784 0.9887192  0.06375232
 0.         0.19119403 0.7886419  0.03302881 0.7024206  0.01970348
 0.8976211  0.9887256  0.21663937 1.5466546  0.17757744 0.09446889
 0.         2.2011735  0.         2.3597434  1.3306153  0.
 0.347929   0.21357715 0.6357652  0.53062326 0.         0.
 0.         2.0496793  0.         0.         0.         0.
 0.24603364 0.6679578  0.397514   1.3602496  0.7451788  0.
 0.3102901  1.1098285  1.6382891  0.         0.35859948 0.
 0.         0.37080052 0.         0.         0.         0.
 0